In [1]:
import logging
import re
import os

# Function to initialize logging with user-specific settings
def initialize_logger(user_name, log_directory='D:\\Bizmetric_Internship\\Logs'):
    os.makedirs(log_directory, exist_ok=True)
    log_path = os.path.join(log_directory, f'{user_name}_logfile.log')
    logger = logging.getLogger(user_name)
    logger.setLevel(logging.DEBUG)

    file_handler = logging.FileHandler(log_path)
    console_handler = logging.StreamHandler()

    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    console_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    logger.info(f"Logger initialized for {user_name} at {log_path}")
    return logger

class UniversityAdmissionSystem:
    def __init__(self, log):
        self.log = log
        self.applicants = []
        self.available_departments = []
    
    def validate_applicant_details(self, full_name, email_address, phone_number, gender_identity, birth_date, grade10, grade12):
        try:
            if not full_name or not isinstance(full_name, str) or len(full_name.split()) < 2:
                self.log.error("Invalid full name provided")
                raise ValueError("Full name must include both first and last name")
            
            email_pattern = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
            if not re.match(email_pattern, email_address):
                self.log.error(f"Invalid email format: {email_address}")
                raise ValueError("Email is not in a valid format")
            
            if not phone_number.isdigit() or len(phone_number) != 10:
                self.log.error(f"Invalid phone number: {phone_number}")
                raise ValueError("Phone number must be 10 digits")
            
            valid_genders = {'Male', 'Female', 'Other'}
            if gender_identity.capitalize() not in valid_genders:
                self.log.error(f"Invalid gender: {gender_identity}")
                raise ValueError("Gender must be Male, Female, or Other")
            
            dob_pattern = r"^\d{2}[-/]\d{2}[-/]\d{4}$"
            if not re.match(dob_pattern, birth_date):
                self.log.error(f"Invalid date of birth format: {birth_date}")
                raise ValueError("Date of birth must be in DD-MM-YYYY or DD/MM/YYYY format")
            
            if not (0 <= grade10 <= 100) or not (0 <= grade12 <= 100):
                self.log.error(f"Invalid marks: 10th={grade10}, 12th={grade12}")
                raise ValueError("Marks must be between 0 and 100")
            
            self.log.info(f"Validation successful for {full_name}")
            return True
        except Exception as e:
            self.log.error(f"Validation error: {e}")
            return False

    def register_applicant(self, full_name, email_address, phone_number, address, gender_identity, birth_date, grade10, grade12):
        if self.validate_applicant_details(full_name, email_address, phone_number, gender_identity, birth_date, grade10, grade12):
            applicant_id = len(self.applicants) + 1
            self.applicants.append({
                'ApplicantID': applicant_id,
                'FullName': full_name,
                'Email': email_address,
                'PhoneNumber': phone_number,
                'Address': address,
                'Gender': gender_identity,
                'DateOfBirth': birth_date,
                'Grade10': grade10,
                'Grade12': grade12
            })
            self.log.info(f"Applicant registered successfully: {full_name}")
        else:
            self.log.warning("Applicant registration failed due to validation issues")

    def add_department(self, dept_name):
        if dept_name not in self.available_departments:
            self.available_departments.append({'DepartmentName': dept_name, 'SeatAvailable': True})
            self.log.info(f"Department added: {dept_name}")
        else:
            self.log.warning(f"Department {dept_name} already exists")

    def assign_seat(self, applicant_id, preferences):
        applicant = next((a for a in self.applicants if a['ApplicantID'] == applicant_id), None)
        if not applicant:
            self.log.error(f"Applicant with ID {applicant_id} not found")
            return

        avg_score = (applicant['Grade10'] + applicant['Grade12']) / 2
        if avg_score < 55:
            self.log.warning(f"Applicant {applicant_id} does not meet the minimum score criteria")
            return

        for preference in preferences:
            department = next((d for d in self.available_departments if d['DepartmentName'] == preference and d['SeatAvailable']), None)
            if department:
                department['SeatAvailable'] = False
                self.log.info(f"Seat allocated to Applicant {applicant_id} in {preference}")
                return

        self.log.warning(f"No available seats for Applicant {applicant_id}'s preferences")

def collect_input():
    user_name = input("Enter your username: ")
    logger = initialize_logger(user_name)

    full_name = input("Enter your full name: ")
    email = input("Enter your email: ")
    phone = input("Enter your phone number: ")
    address = input("Enter your address: ")
    gender = input("Enter your gender (Male/Female/Other): ")
    dob = input("Enter your date of birth (DD-MM-YYYY or DD/MM/YYYY): ")
    grade_10 = float(input("Enter your 10th-grade percentage: "))
    grade_12 = float(input("Enter your 12th-grade percentage: "))

    print("Available courses:")
    course_options = {
        1: 'Civil Engineering',
        2: 'Information Technology',
        3: 'Electrical Engineering',
        4: 'Computer Engineering',
        5: 'Mechanical Engineering',
        6: 'Electronics Engineering'
    }
    for key, value in course_options.items():
        print(f"{key}. {value}")

    preferences = input("Enter course preferences as numbers, separated by commas: ").split(",")
    chosen_courses = [course_options[int(num.strip())] for num in preferences if int(num.strip()) in course_options]

    return user_name, logger, full_name, email, phone, address, gender, dob, grade_10, grade_12, chosen_courses

if __name__ == "__main__":
    user_name, logger, full_name, email, phone, address, gender, dob, grade_10, grade_12, courses = collect_input()
    university = UniversityAdmissionSystem(logger)

    university.register_applicant(full_name, email, phone, address, gender, dob, grade_10, grade_12)

    departments = [
        'Civil Engineering', 'Information Technology', 'Electrical Engineering',
        'Computer Engineering', 'Mechanical Engineering', 'Electronics Engineering'
    ]
    for dept in departments:
        university.add_department(dept)

    university.assign_seat(1, courses)


2025-01-22 22:17:49,427 - INFO - Logger initialized for ruchitaurkude at D:\Bizmetric_Internship\Logs\ruchitaurkude_logfile.log


Available courses:
1. Civil Engineering
2. Information Technology
3. Electrical Engineering
4. Computer Engineering
5. Mechanical Engineering
6. Electronics Engineering


2025-01-22 22:18:47,078 - ERROR - Invalid full name provided
2025-01-22 22:18:47,078 - ERROR - Validation error: Full name must include both first and last name
2025-01-22 22:18:47,083 - WARNING - Applicant registration failed due to validation issues
2025-01-22 22:18:47,087 - INFO - Department added: Civil Engineering
2025-01-22 22:18:47,087 - INFO - Department added: Information Technology
2025-01-22 22:18:47,087 - INFO - Department added: Electrical Engineering
2025-01-22 22:18:47,087 - INFO - Department added: Computer Engineering
2025-01-22 22:18:47,099 - INFO - Department added: Mechanical Engineering
2025-01-22 22:18:47,099 - INFO - Department added: Electronics Engineering
2025-01-22 22:18:47,107 - ERROR - Applicant with ID 1 not found
